In [1]:
import pandas as pd
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
df = pd.read_csv("./legal-act-classification/train_title.csv")
committee = pd.read_csv("./legal-act-classification/committee.csv")
pattern = pd.read_csv("./legal-act-classification/patterns.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4429 entries, 0 to 4428
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4429 non-null   int64  
 1   rgno       4429 non-null   float64
 2   context    4429 non-null   object 
 3   pattern    4429 non-null   int64  
 4   question   4429 non-null   object 
 5   legal_act  4428 non-null   object 
 6   condition  428 non-null    object 
 7   answer     4429 non-null   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 276.9+ KB


In [5]:
from tqdm.auto import tqdm

def preprocess(df):
    x = []
    y = []

    for i in tqdm(range(len(df))):
        cmt = {}
        label = df['answer'][i]
        count = 0
        for _, row in committee[committee["rgno"] == df['rgno'][i]].iterrows():
            cmt["{}{} {}".format(row["title"], row["fname"], row["lname"])] = count
            cmt["{} {} {}".format(row["title"], row["fname"], row["lname"])] = count
            count += 1

        context = df["context"][i]
        question = df["question"][i]
        for i in cmt:
            if i in context:
                context = context.replace(i, f"<PERSON{cmt[i]}>")
            if i in question:
                question = question.replace(i, f"<PERSON{cmt[i]}>")

        txt = context + " <mask> " + question
        x.append(txt)
        y.append(label)
    
    return x, y

In [6]:
x, y = preprocess(df)

  0%|          | 0/4429 [00:00<?, ?it/s]

In [7]:
print(x[500])

<PERSON0> หรือ <PERSON1>หรือ <PERSON2> หรือ <PERSON3> หรือ <PERSON4> ลงลายมือชื่อร่วมกับ<PERSON5> หรือ <PERSON6> หรือ <PERSON7> รวมเป็นสองคน <mask> ['<PERSON2>', '<PERSON7>']


In [8]:
print(y[500])

0


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "clicknext/phayathaibert"
tokenizer_name = "clicknext/phayathaibert"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                          num_labels=2,
                                                          ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at clicknext/phayathaibert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import datasets

dsf = pd.DataFrame(x, columns=["text"])
dsf["labels"] = y
dsf.head(10)

,text,labels
0,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0
1,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0
2,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0
3,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0
4,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0
5,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0
6,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,0
7,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,1
8,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,1
9,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,1


In [12]:
ds = datasets.Dataset.from_pandas(dsf[["text", "labels"]])
ds

Dataset({
    features: ['text', 'labels'],
    num_rows: 4429
})

In [15]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def tokenization(example):
    return tokenizer(example["text"], truncation=True, padding='max_length', max_length=384, return_tensors='pt')

In [16]:
ds = ds.map(tokenization, batched=True)

Map:   0%|          | 0/4429 [00:00<?, ? examples/s]

In [18]:
train_ds, test_ds = ds.train_test_split(test_size=0.1, shuffle=True, seed=1234).values()

In [20]:
import evaluate

metric = evaluate.load("accuracy")
f1 = evaluate.load("f1")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    f1_score = f1.compute(predictions=predictions, references=labels)
    recall_score = recall.compute(predictions=predictions, references=labels)
    acc_score = metric.compute(predictions=predictions, references=labels)
    return {'accuracy': acc_score['accuracy'], 'recall': recall_score['recall'], 'f1': f1_score['f1']}

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="my-model",
    overwrite_output_dir = 'True',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps = 100,
    load_best_model_at_end=True,
)

In [22]:
from transformers import Trainer
import numpy as np

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [23]:
trainer.train()

../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [324,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [324,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [324,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [324,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [324,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [324,0,0], thread: [101,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [32

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [37]:
import numpy as np

trainer.evaluate(test_ds)

{'eval_loss': 0.17089080810546875,
 'eval_accuracy': 0.9548532731376975,
 'eval_f1': 0.9319727891156463,
 'eval_runtime': 2.0258,
 'eval_samples_per_second': 218.678,
 'eval_steps_per_second': 13.822,
 'epoch': 10.0}

# Submission

In [38]:
test_df = pd.read_csv("./legal-act-classification/test_title.csv")
test_df.head()

,id,rgno,context,pattern,question,legal_act,condition
0,0,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['นางสาว สหชา อินทร์ไชย'],การขอวีซ่า,NaN
1,1,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['นาย สุเมธ คุ้มคง'],การค้ำประกัน,NaN
2,2,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['นาย ภควัฒน์ ผาสุข'],ใบอนุญาตจำหน่ายสุราและจำหน่ายยาสูบ,NaN
3,3,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['นาย อชิระ อาจนาเสียว'],ธนาคาร,NaN
4,4,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['นางสาว ณัฐวีร์ แนวกำพล'],มัคคุเทศน์,NaN


In [39]:
def preprocess_test(df):
    x = []

    for i in tqdm(range(len(df))):
        cmt = {}
        count = 0
        for _, row in committee[committee["rgno"] == df['rgno'][i]].iterrows():
            cmt["{}{} {}".format(row["title"], row["fname"], row["lname"])] = count
            cmt["{} {} {}".format(row["title"], row["fname"], row["lname"])] = count
            count += 1

        context = df["context"][i]
        question = df["question"][i]
        for i in cmt:
            if i in context:
                context = context.replace(i, f"<PERSON{cmt[i]}>")
            if i in question:
                question = question.replace(i, f"<PERSON{cmt[i]}>")

        txt = context + ". " + question
        x.append(txt)
    
    return x

In [40]:
test = preprocess_test(test_df)

  0%|          | 0/5835 [00:00<?, ?it/s]

In [41]:
print(test[0])

ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป็นสำคัญผูกพันบริษัทได้. ['<PERSON0>']


In [43]:
from transformers import pipeline

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device="cuda", batch_size=8)

In [44]:
results = []
for i in tqdm(range(0, len(test), nlp.model.config.max_position_embeddings)):
    batch = test[i:i+nlp.model.config.max_position_embeddings]
    batch_results = nlp(batch)
    results.extend(batch_results)

print(results[0])

  0%|          | 0/12 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'label': 'LABEL_1', 'score': 0.9987047910690308}


In [45]:
len(results)

5835

In [46]:
def prediction(pred):
    
    return 0 if pred["label"] == "LABEL_0" else 1

In [47]:
y_pred = []

for i in tqdm(results):
    y_pred.append(prediction(i))
    
y_pred[:20]

  0%|          | 0/5835 [00:00<?, ?it/s]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [48]:
submit = pd.read_csv("./legal-act-classification/sample_submission.csv")
submit.head()

,id,answer
0,0,1.0
1,1,1.0
2,2,1.0
3,3,NaN
4,4,NaN


In [49]:
submit["answer"] = y_pred

In [50]:
submit.head()

,id,answer
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [52]:
submit.to_csv("submission3.csv", index=False)

# Evaluate

In [65]:
evaluate = pd.read_csv("./Evaluate/tongdata2.csv")
evaluate["answer"] = evaluate["Answer"]
evaluate_text, evaluate_labels = preprocess(evaluate)
edfe = pd.DataFrame({"text": evaluate_text, "labels": evaluate_labels})
data_eval = level1_eval.map(tokenization, batched=True)

  0%|          | 0/1810 [00:00<?, ?it/s]

Map:   0%|          | 0/721 [00:00<?, ? examples/s]

In [66]:
trainer.evaluate(data_eval)

{'eval_loss': 0.2861391603946686,
 'eval_accuracy': 0.9181692094313454,
 'eval_f1': 0.9502948609941028,
 'eval_runtime': 3.2965,
 'eval_samples_per_second': 218.718,
 'eval_steps_per_second': 13.954,
 'epoch': 10.0}

In [67]:
evaluate["answer"].value_counts()

answer
1    1429
0     381
Name: count, dtype: int64

In [92]:
eval_level1 = evaluate[evaluate["pattern"].astype(str).str.startswith("1")]
eval_level2 = evaluate[evaluate["pattern"].astype(str).str.startswith("2")]
eval_level1.reset_index(inplace = True, drop = True)
eval_level2.reset_index(inplace = True, drop = True)

In [94]:
eval_level1_text, eval_level1_labels = preprocess(eval_level1)
eval_level2_text, eval_level2_labels = preprocess(eval_level2)
edfe1 = pd.DataFrame({"text": eval_level1_text, "labels": eval_level1_labels})
edfe2 = pd.DataFrame({"text": eval_level2_text, "labels": eval_level2_labels})

  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/695 [00:00<?, ?it/s]

In [98]:
edfe1

,text,labels
0,<PERSON0> หรือ<PERSON1> หรือ <PERSON2> คนใดคนห...,0
1,<PERSON0> หรือ<PERSON1> หรือ <PERSON2> คนใดคนห...,0
2,<PERSON0> หรือ<PERSON1> หรือ <PERSON2> คนใดคนห...,0
3,<PERSON0> หรือ<PERSON1> หรือ <PERSON2> คนใดคนห...,0
4,<PERSON0> หรือ<PERSON1> หรือ <PERSON2> คนใดคนห...,0
...,...,...
1110,กรรมการสองคนลงลายมือชื่อร่วมกันและประทับตราสำค...,1
1111,กรรมการสองคนลงลายมือชื่อร่วมกันและประทับตราสำค...,1
1112,กรรมการสองคนลงลายมือชื่อร่วมกันและประทับตราสำค...,1
1113,กรรมการสองคนลงลายมือชื่อร่วมกันและประทับตราสำค...,1


In [96]:
level1_eval = datasets.Dataset.from_pandas(edfe1[["text", "labels"]])
level2_eval = datasets.Dataset.from_pandas(edfe2[["text", "labels"]])

level1_eval = level1_eval.map(tokenization, batched=True)
level2_eval = level2_eval.map(tokenization, batched=True)

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Map:   0%|          | 0/695 [00:00<?, ? examples/s]

In [97]:
trainer.evaluate(edfe1)

KeyError: 0

In [87]:
test_ds["labels"].count()

303